In [ ]:
!pip install pypdf

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index

In [ ]:
%pip install llama-index-llms-huggingface

In [ ]:
!pip install llama-index[prompts]

In [ ]:
#from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContent
#from llama_index.llms import HuggingFaceLLM
#from llama_index.prompts.prompts import SimpleInputPrompt

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

In [ ]:
data=SimpleDirectoryReader('/content/data').load_data()
data

[Document(id_='fba24203-7c2e-4954-995b-4c28e2661a89', embedding=None, metadata={'page_label': '1', 'file_name': '1409.3215.pdf', 'file_path': '/content/data/1409.3215.pdf', 'file_type': 'application/pdf', 'file_size': 112084, 'creation_date': '2024-04-08', 'last_modified_date': '2024-04-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='arXiv:1409.3215v3  [cs.CL]  14 Dec 2014Sequenceto SequenceLearning\nwithNeuralNetworks\nIlyaSutskever\nGoogle\nilyasu@google.comOriolVinyals\nGoogle\nvinyals@google.comQuocV.Le\nGoogle\nqvl@google.com\nAbstract\nDeep Neural Networks (DNNs) are powerful models that have ac hieved excel-\nlentperformanceondifﬁcultlearningtasks. AlthoughDNNs workwellwhenever\nlarge labeled training sets are available, they cannot be us ed

In [ ]:
System_prompt="""
You are Q&A assitant .your goal is to answer questions as
accurately as possible based on the instructions and context provided
"""
##default format supportable by llama2
query_wrapper_prompt=PromptTemplate("<|USER|>{query_str}<|ASSISTANT|")

In [ ]:
!!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer_name = model_name

llm = HuggingFaceLLM(model_name=model_name,
                     tokenizer_name=tokenizer_name,
                     context_window=4096,
                     max_new_tokens=256,
                     device_map="auto",
                     query_wrapper_prompt=query_wrapper_prompt,
                     system_prompt=System_prompt,
                     generate_kwargs={'temperature':0.0,"do_sample":False},

                     )


In [ ]:
%pip install llama-index-embeddings-langchain

service_content is replace by settings

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embedding
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

In [ ]:
# a document summary index needs both an llm and embed model
# for the constructor
index = DocumentSummaryIndex.from_documents(
    documents, embed_model=embed_model, llm=llm
)

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
query_engine.query("what is mean by bert model ?")